# Homework 2

## 1.1  Create the spam filter

In [1]:
import numpy

def unique_counter(unique_list, corpus):
    '''
    add unique words from a corpus to a list
    '''
    for sample in corpus:
        for word in sample:
            # check if word in this sample has already been added
            if word not in unique_list:
                unique_list.append(word)
    return unique_list

def word_hash(corpus):
    '''
    create a hash for each word in a corpus of the number of times it appears
    '''
    count_hash = {}
    for sample in corpus:
        done = []
        for word in sample:
            # check if word in this sample has already been counted
            if word not in done:
                # check if word already has an entry and if so
                # update the new count
                if word in count_hash:
                    count_hash[word] = sample.count(word) + count_hash[word]
                else:
                    count_hash[word] = sample.count(word)
                done.append(word)
    return count_hash

def word_prob(good, bad, word, ngood, nbad):
    '''
    compute the probability that a word is spam
    '''
    g = 2 * good.get(word, 0)
    b = bad.get(word, 0)
    if g + b >= 1:
        return max(0.01, min(0.99, min(1.0, b / nbad) / (min(1.0, g / ngood) + min(1.0, b / nbad))))
    return -1


def prob_hash(good, bad, ngood, nbad, word_list):
    '''
    find the probability a word means something is spam and put this value in a hash
    '''
    spam_prob = {}
    for word in word_list:
        spam_prob[word] = word_prob(good, bad, word, ngood, nbad)
    return spam_prob


def is_mail_spam_probability(mail_corpus, prob_hash):
    '''
    a way to check if new mail is spam
    assumes the mail comes in the form of a list of tokens
    '''
    # Get all the probabilities in the mail corpus
    # (Ignoring the self imposed 15 word limit due to average short length)
    # if word not in the probs hash then assign it .4
    probs = []
    for word in mail_corpus:
        if word in prob_hash:
            probs.append(prob_hash[word])
        else:
            probs.append(0.4)
    # Get combined probability
    compliment = []
    for i in range(len(probs)):
        compliment.append(1 - probs[i])
    combined = numpy.prod(probs) / (numpy.prod(probs) + numpy.prod(compliment))
    if combined > 0.9:
        print("Probability of being spam:", str(combined))
        return True
    else:
        print("Probability of being spam:", str(combined))
        return False

# a corpus has to a be a list of lists which contain strings
spam_corpus = [["I", "am", "spam", "spam", "I", "am"], ["I", "do", "not", "like", "that", "spamiam"]]
ham_corpus = [["do", "i", "like", "green", "eggs", "and", "ham"], ["i", "do"]]

# get unique words in a each corpus
word_list = []
word_list = unique_counter(word_list, ham_corpus)
word_list = unique_counter(word_list, spam_corpus)

# get count hashtable for each corpus
spam_hash = word_hash(spam_corpus)
ham_hash = word_hash(ham_corpus)
# get size of each
ngood = len(ham_corpus)
nbad = len(spam_corpus)

# generate a  probability table an email with that word is spam
spam_prob_hash = prob_hash(ham_hash, spam_hash, ngood, nbad, word_list)
print("Probability table of words after loading in corpus:", str(spam_prob_hash))

# test with new messages
test = ["This", "is", "a", "clean", "message", "for", "enjoyment"]
test2 = ["I", "am", "very", "much", "spam", "spam",  "I", "very", "much", "am"]
print(is_mail_spam_probability(test, spam_prob_hash))
print(is_mail_spam_probability(test2, spam_prob_hash))


Probability table of words after loading in corpus: {'do': 0.3333333333333333, 'i': 0.01, 'like': 0.3333333333333333, 'green': 0.01, 'eggs': 0.01, 'and': 0.01, 'ham': 0.01, 'I': 0.99, 'am': 0.99, 'spam': 0.99, 'not': 0.99, 'that': 0.99, 'spamiam': 0.99}
Probability of being spam: 0.05529157667386612
False
Probability of being spam: 0.9999999999946227
True


## 1.2 Why is this implimentation Bayesian?
This implementation of the spam filter is Bayesian because it uses past calculated information 
and probabilities to calculate the probability a new message is spam.
This algorithm does not just consider the words with a high spam probability (like many spam feature
recognizing systems) but also the ones that have a low spam probability. Using the overall probability 
is more Bayesian. It is also Bayesian in giving a new probability (that the email is spam)
by combining other probabilities (the word within it indicate it is spam).


## 2.a Implement Figure 14.12a Bayesian Network

In [2]:
from probability import BayesNet, enumeration_ask

# Utility variables
T, F = True, False

wet = BayesNet([
    ('Cloudy', '', 0.5),
    ('Sprinkler', 'Cloudy', {T: 0.1, F: 0.50}),
    ('Rain', 'Cloudy', {T: 0.80, F: 0.2}),
    ('WetGrass', 'Sprinkler Rain', {(T, T): 0.99, (T, F): 0.90, (F, T): 0.90, (F, F): 0.00})
    ])

# Compute P(cloudy)
print(enumeration_ask('Cloudy', dict(), wet).show_approx())

# Compute P(Sprinkler | cloudy)
print(enumeration_ask('Sprinkler', dict(Cloudy=T), wet).show_approx())

# Compute P(Cloudy| the sprinkler is running and it’s not raining)
print(enumeration_ask('Cloudy', dict(Sprinkler=T, Rain=F), wet).show_approx())

# Compute P(WetGrass | it’s cloudy, the sprinkler is running and it’s raining)
print(enumeration_ask('WetGrass', dict(Cloudy=T, Sprinkler=T, Rain=T), wet).show_approx())

# Compute P(Cloudy | the grass is not wet)
print(enumeration_ask('Cloudy', dict(WetGrass=F), wet).show_approx())

False: 0.5, True: 0.5
False: 0.9, True: 0.1
False: 0.952, True: 0.0476
False: 0.01, True: 0.99
False: 0.639, True: 0.361


## 2.b Compute the number of independent values in the full joint probability
 There are 4 binary variables in this system meaning that the number of values
 should be 2^4 = 16

## 2.c Compute the number of independent values in the Bayesian network 
Based on the Bayesian network figure, it will have 9 values.

## 2.d Compute probabilities by hand
- P(Cloudy) = <T=.5,F=.5> (given in text)

- P(Sprinker | cloudy) <0.1,0.9> (given in text)

- P(Cloudy| sprinkler, ¬rain) = α P(cloudy, sprinkler, ¬rain) = <br>
  α <P(S|C)* P(¬R|C)* (P(C), P(S|¬C)* P(¬R|¬C)* P(¬C)> =
  α<.1* .2* .5 , .5* .8* .5> = α<.01 , .2> = <.048 , .952>


- P(WetGrass | cloudy, sprinkler, rain) = α P(wetgrass, cloudy, sprinkler, rain)
  α <P(W | S,R) * P(S|C) * P(R|C) * P(C) , P(¬W | S,R) * P(S|C) * P(R|C) * P(C)> = 
  α < .99* .10* .80* .5 , .01* .10* .80* .5> = α <.0396 , .0004 > = <.99 , .01>

- P(Cloudy | ¬wetgrass) = α P(Cloudy,¬wetgrass, rain, sprinkler)<br>
  α <P(C) * P(R|C) * P(S|C)* P(¬W | S,R) + P(C) * P(¬R|C) * P(S|C) * P(¬W | S,¬R) <br>
  +P(C) * P(R|C) * P(¬S|C) * P(¬W | ¬S,R)  + P(C) * P(¬R|C) * P(¬S|C) * P(¬W | ¬S,¬R) , <br>
  P(¬C) * P(R|¬C) * P(S|¬C) * P(¬W | S,R) + P(¬C) * P(¬R|¬C) * P(S|¬C) * P(¬W | S,¬R) <br>
  +P(¬C) * P(R|¬C) * P(¬S|¬C) * P(¬W | ¬S,R) +P(¬C) * P(¬R|¬C) * P(¬S|¬C) * P(¬W | ¬S,¬R)><br>
  α <.5* .8* .1* .01 + .5* .2* .1* .1 + .5* .8* .9* .1 + .5* .2* .9* 1 ,
  .5 *.2 *.5 *.01 + .5 *.8 *.5 *.1 + .5 *.2 *.5 *.1 + .5 *.8 *.5 *1><br>
  α <.1272,.2255> = <.361,.639>